In [1]:
from hoi.datasets import H2ODataset
from collections import Counter

In [2]:
dataset = H2ODataset("datasets/h2o/")

In [10]:
different_subjects_same_target = 0
same_subject_different_targets = 0
total_actions = 0
total_interactions = 0
has_instrument = 0
same_subject_target = 0

dsst_samples = []
ssdt_samples = []
at_least_one = []
both = []

for _, sample in dataset:
    actions: dict[str, list[tuple[int, int | None]]] = {}
    for action in sample.actions:
        total_actions += 1
        if action.instrument is not None:
            has_instrument += 1
        
        pairs = actions.get(action.verb, [])
        pairs.append((action.subject, action.target))
        actions[action.verb] = pairs
        if action.target is not None:
            total_interactions += 1

    has_dsst = False
    has_ssdt = False
    
    for pairs in actions.values():
        subject_counter = Counter([pair[0] for pair in pairs if pair[1] is not None])
        target_counter = Counter([pair[1] for pair in pairs if pair[1] is not None])

        same_subject_target += sum([1 for subject in subject_counter if subject in target_counter])
    
        for subject, count in subject_counter.items():
            if count > 1:
                same_subject_different_targets += 1
                has_ssdt = True
        
        for target, count in target_counter.items():
            if count > 1:
                different_subjects_same_target += 1
                has_dsst = True
        
        if has_dsst:
            dsst_samples.append(sample)
        if has_ssdt:
            ssdt_samples.append(sample)
        
        if has_dsst and has_ssdt:
            both.append(sample)
        if has_dsst or has_ssdt:
            at_least_one.append(sample)

In [11]:
total_actions, total_interactions, has_instrument, different_subjects_same_target, same_subject_different_targets, same_subject_target

(141755, 48343, 1610, 2505, 1697, 2921)

In [9]:
len(dataset), len(dsst_samples), len(ssdt_samples), len(both), len(at_least_one)

(13896, 6061, 3738, 741, 9058)